# Projet sur l'effectifs d'étudiants

In [1]:
import pandas as pd
import pymongo

In [2]:
from pymongo import MongoClient

client = MongoClient("mongo")

database = client.yt

collection = database['etudiant']

### Nettoyer les données

In [3]:
df_students = pd.read_csv("./data/data.csv")
df_students.head(2)

l_d =df_students.to_dict(orient='records')

index = 1

import math
from math import isnan

for row in l_d:
    index += 1
    row["_id"] = index 
    
    if math.isnan(row["effectif"]):
        row["effectif"] = 0
    else:
        row['effectif'] = int(row['effectif'])
        
    if math.isnan(row["effectifhdccpge"]):
        row["effectifhdccpge"] = 0
    else:
        row['effectifhdccpge'] = int(row['effectifhdccpge'])
        
    if math.isnan(row["effectif_dut"]):
        row["effectif_dut"] = 0
    else:
        row['effectif_dut'] = int(row['effectif_dut'])
        
    if math.isnan(row["effectif_ing"]):
        row["effectif_ing"] = 0
    else:
        row['effectif_ing'] = int(row['effectif_ing'])

,rentree,rentree_universitaire,niveau_geographique,geo_nom,regroupement,rgp_formations_ou_etablissements,secteur,secteur_de_l_etablissement,sexe,sexe_de_l_etudiant,...,diffusable,donnees_diffusables,secret,donnees_soumises_au_secret_stat,niveau_geo,geo_id,reg_id,aca_id,dep_id,uucr_id
0,2017,2017-18,Unité urbaine,Le Robert,STS,Sections de techniciens supérieurs (STS) et as...,PU,Établissements publics,1,Masculin,...,oui,Données diffusables,non,Autres données,UNITE_URBAINE,UU9B602,R02,NaN,NaN,UU9B602
1,2017,2017-18,Unité urbaine,Mana,STS,Sections de techniciens supérieurs (STS) et as...,PU,Établissements publics,2,Feminin,...,oui,Données diffusables,non,Autres données,UNITE_URBAINE,UU9C203,R03,NaN,NaN,UU9C203


## Importer les données

In [4]:
collection.delete_many({})

collection.insert_many(l_d)

### Question 1 : Calculer le nombre d'étudiants par rentrée

In [5]:
cur = collection.aggregate([{"$group":{"_id":"$rentree", "totalEffectif" : {"$sum":"$effectif"}}},{"$sort":{"_id":-1}}])
list(cur)

[{'_id': 2018, 'totalEffectif': 32723237},
 {'_id': 2017, 'totalEffectif': 32051372},
 {'_id': 2016, 'totalEffectif': 31324945},
 {'_id': 2015, 'totalEffectif': 30572101},
 {'_id': 2014, 'totalEffectif': 29630547},
 {'_id': 2013, 'totalEffectif': 29236656},
 {'_id': 2012, 'totalEffectif': 28586765},
 {'_id': 2011, 'totalEffectif': 28223165},
 {'_id': 2010, 'totalEffectif': 27876979},
 {'_id': 2009, 'totalEffectif': 27773179},
 {'_id': 2008, 'totalEffectif': 26854944},
 {'_id': 2007, 'totalEffectif': 26819355},
 {'_id': 2006, 'totalEffectif': 27070261},
 {'_id': 2005, 'totalEffectif': 27439086},
 {'_id': 2004, 'totalEffectif': 27285822},
 {'_id': 2003, 'totalEffectif': 27120742},
 {'_id': 2002, 'totalEffectif': 26540722},
 {'_id': 2001, 'totalEffectif': 26009538}]

### Question 2 : Afficher l'effectif masculin et féminin par rentrée

In [6]:
cur = collection.aggregate([{"$group":{"_id":{"rentree":"$rentree", "sexe":"$sexe_de_l_etudiant"}, "effectif" : {"$sum": "$effectif"}}},{"$sort":{"_id":-1}}]) 
list(cur)

[{'_id': {'rentree': 2018, 'sexe': 'Masculin'}, 'effectif': 14680464},
 {'_id': {'rentree': 2018, 'sexe': 'Feminin'}, 'effectif': 18042773},
 {'_id': {'rentree': 2017, 'sexe': 'Masculin'}, 'effectif': 14395372},
 {'_id': {'rentree': 2017, 'sexe': 'Feminin'}, 'effectif': 17656000},
 {'_id': {'rentree': 2016, 'sexe': 'Masculin'}, 'effectif': 14059879},
 {'_id': {'rentree': 2016, 'sexe': 'Feminin'}, 'effectif': 17265066},
 {'_id': {'rentree': 2015, 'sexe': 'Masculin'}, 'effectif': 13714626},
 {'_id': {'rentree': 2015, 'sexe': 'Feminin'}, 'effectif': 16857475},
 {'_id': {'rentree': 2014, 'sexe': 'Masculin'}, 'effectif': 13245633},
 {'_id': {'rentree': 2014, 'sexe': 'Feminin'}, 'effectif': 16384914},
 {'_id': {'rentree': 2013, 'sexe': 'Masculin'}, 'effectif': 13054730},
 {'_id': {'rentree': 2013, 'sexe': 'Feminin'}, 'effectif': 16181926},
 {'_id': {'rentree': 2012, 'sexe': 'Masculin'}, 'effectif': 12753324},
 {'_id': {'rentree': 2012, 'sexe': 'Feminin'}, 'effectif': 15833441},
 {'_id': {'re

### Question 3 : Compter le nombre d'établissements

In [7]:
cur = collection.aggregate([{"$group":{"_id":"$uucr_id"}},{"$count":"uucr_id"}])
list(cur)

[{'uucr_id': 750}]

### Question 4 : Afficher les établissements pour la rentrée de 2017-2018

In [8]:
cur = collection.find({"rentree_universitaire" : "2017-18"})
list(cur)

[{'_id': 2,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Le Robert',
  'regroupement': 'STS',
  'rgp_formations_ou_etablissements': 'Sections de techniciens supérieurs (STS) et assimilés',
  'secteur': 'PU',
  'secteur_de_l_etablissement': 'Établissements publics',
  'sexe': 1,
  'sexe_de_l_etudiant': 'Masculin',
  'effectif': 134,
  'effectifhdccpge': 0,
  'a_des_effectifs_dut': 'non',
  'effectif_dut': 0,
  'a_des_effectifs_ing': 'non',
  'effectif_ing': 0,
  'diffusable': 'oui',
  'donnees_diffusables': 'Données diffusables',
  'secret': 'non',
  'donnees_soumises_au_secret_stat': 'Autres données',
  'niveau_geo': 'UNITE_URBAINE',
  'geo_id': 'UU9B602',
  'reg_id': 'R02',
  'aca_id': nan,
  'dep_id': nan,
  'uucr_id': 'UU9B602'},
 {'_id': 3,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Mana',
  'regroupement': 'STS',
  'rgp_formations_ou_etablisseme

### Question 5 : Afficher la rentrée la moins récente

In [9]:
cur = collection.aggregate([{"$group":{"_id":"", "oldestSchoolYear":{"$min":"$rentree"}}}])
oldestSchoolYear = next(cur)["oldestSchoolYear"]
print(oldestSchoolYear)

2001


### Question 6 : Supprimer toute les données de la rentrée la moins récente

In [10]:
cur = collection.delete_many({"rentree":oldestSchoolYear}) 
print(cur)

### Question 7 : Ajouter un établissement 

In [11]:
collection.insert_one({"_id":123456789, "rentree":2042, "geo_nom":"test"})
list(collection.find({"rentree":2042}))

[{'_id': 123456789, 'rentree': 2042, 'geo_nom': 'test'}]

### Question 8 : Rechercher tous les établissement avec une formation : "spécialité"

In [12]:
import re

rgx = re.compile('.spécialité.', re.IGNORECASE)

cur = collection.find({"rgp_formations_ou_etablissements":rgx})
list(cur)

[{'_id': 6,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Cayenne',
  'regroupement': 'EC_autres',
  'rgp_formations_ou_etablissements': 'Autres écoles de spécialités diverses',
  'secteur': 'PU',
  'secteur_de_l_etablissement': 'Établissements publics',
  'sexe': 2,
  'sexe_de_l_etudiant': 'Feminin',
  'effectif': 24,
  'effectifhdccpge': 0,
  'a_des_effectifs_dut': 'non',
  'effectif_dut': 0,
  'a_des_effectifs_ing': 'non',
  'effectif_ing': 0,
  'diffusable': 'oui',
  'donnees_diffusables': 'Données diffusables',
  'secret': 'non',
  'donnees_soumises_au_secret_stat': 'Autres données',
  'niveau_geo': 'UNITE_URBAINE',
  'geo_id': 'UU9C601',
  'reg_id': 'R03',
  'aca_id': nan,
  'dep_id': nan,
  'uucr_id': 'UU9C601'},
 {'_id': 7,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Cayenne',
  'regroupement': 'EC_autres',
  'rgp_formations_ou_etablissements':

### Question 9 : Afficher les établissements ayant pour regroupement TOTAL pour la rentrée 2018-19

In [13]:
cur = collection.find({"$and":[{"rentree_universitaire":"2017-18"},{"regroupement":"TOTAL"}]})
list(cur)

[{'_id': 8,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Les Trois-Bassins',
  'regroupement': 'TOTAL',
  'rgp_formations_ou_etablissements': "Total des formations d'enseignement supérieur",
  'secteur': 'PU',
  'secteur_de_l_etablissement': 'Établissements publics',
  'sexe': 1,
  'sexe_de_l_etudiant': 'Masculin',
  'effectif': 50,
  'effectifhdccpge': 0,
  'a_des_effectifs_dut': 'non',
  'effectif_dut': 0,
  'a_des_effectifs_ing': 'non',
  'effectif_ing': 0,
  'diffusable': 'oui',
  'donnees_diffusables': 'Données diffusables',
  'secret': 'non',
  'donnees_soumises_au_secret_stat': 'Autres données',
  'niveau_geo': 'UNITE_URBAINE',
  'geo_id': 'UU9D203',
  'reg_id': 'R04',
  'aca_id': nan,
  'dep_id': nan,
  'uucr_id': 'UU9D203'},
 {'_id': 14,
  'rentree': 2017,
  'rentree_universitaire': '2017-18',
  'niveau_geographique': 'Unité urbaine',
  'geo_nom': 'Saint-Benoît',
  'regroupement': 'TOTAL',
  'rgp_formations_ou

### Question 10 : Afficher les différentes formations

In [14]:
cur = collection.aggregate([{"$group":{"_id":"$rgp_formations_ou_etablissements"}}])
list(cur)

[{'_id': 'Sections de techniciens supérieurs (STS) et assimilés'},
 {'_id': 'ESPE'},
 {'_id': 'Écoles paramédicales et sociales'},
 {'_id': "Etablissements d'enseignement universitaire privés"},
 {'_id': 'Écoles supérieures art et culture'},
 {'_id': 'Ecoles de commerce, gestion et comptabilité'},
 {'_id': "Autres formations d'ingénieurs"},
 {'_id': 'Grands établissements MENESR'},
 {'_id': 'Instituts nationaux polytechniques (INP)'},
 {'_id': None},
 {'_id': 'Écoles normales supérieures (ENS)'},
 {'_id': "Total des formations d'enseignement supérieur"},
 {'_id': 'Universités'},
 {'_id': 'Écoles juridiques et administratives'},
 {'_id': 'Autres écoles de spécialités diverses'},
 {'_id': 'Classes préparatoires aux grandes écoles (CPGE)'},
 {'_id': 'Universités de technologie (UT)'}]

### Question 11 : Afficher l'effectif maximum

In [16]:
cur = collection.aggregate([{"$group":{"_id": "", "maxEffectif": { "$max": "$effectif" }}}])
list(cur)

[{'_id': '', 'maxEffectif': 1228070}]